In [ ]:
import pandas as pd
import numpy as np 
import torch

from function import manipulation, manipulation_cutoff
from causalml.inference.meta import BaseDRRegressor


import matplotlib.pyplot as plt
import seaborn as sns;sns.set()
from matplotlib.ticker import FuncFormatter

from utils import *

import sklearn
import xgboost

sklearn.__version__

/root/miniconda3/envs/myconda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'1.3.0'

In [ ]:
# preprocess

df = pd.read_csv('features_with_pred_prob.csv')
df_loan = df['AMT_CREDIT'].copy()
df_prob = df['PROB_PRED'].copy()

df['AMT_INCOME_TOTAL'] = (df['AMT_INCOME_TOTAL'] - df['AMT_INCOME_TOTAL'].mean()) / df['AMT_INCOME_TOTAL'].std()
df['AMT_CREDIT'] = (df['AMT_CREDIT'] - df['AMT_CREDIT'].mean()) / df['AMT_CREDIT'].std()
df['REGION_POPULATION_RELATIVE'] = (df['REGION_POPULATION_RELATIVE'] - df['REGION_POPULATION_RELATIVE'].mean()) / df['REGION_POPULATION_RELATIVE'].std()
df['DAYS_BIRTH'] = (df['DAYS_BIRTH'] - df['DAYS_BIRTH'].mean()) / df['DAYS_BIRTH'].std()
df['DAYS_EMPLOYED'] = (df['DAYS_EMPLOYED'] - df['DAYS_EMPLOYED'].mean()) / df['DAYS_EMPLOYED'].std()
df['DAYS_REGISTRATION'] = (df['DAYS_REGISTRATION'] - df['DAYS_REGISTRATION'].mean()) / df['DAYS_REGISTRATION'].std()
df['DAYS_ID_PUBLISH'] = (df['DAYS_ID_PUBLISH'] - df['DAYS_ID_PUBLISH'].mean()) / df['DAYS_ID_PUBLISH'].std()
df['DAYS_LAST_PHONE_CHANGE'] = (df['DAYS_LAST_PHONE_CHANGE'] - df['DAYS_LAST_PHONE_CHANGE'].mean()) / df['DAYS_LAST_PHONE_CHANGE'].std()
df['HOUR_APPR_PROCESS_START'] = (df['HOUR_APPR_PROCESS_START'] - df['HOUR_APPR_PROCESS_START'].mean()) / df['HOUR_APPR_PROCESS_START'].std()
df['ORGANIZATION_TYPE'] = (df['ORGANIZATION_TYPE'] - df['ORGANIZATION_TYPE'].mean()) / df['ORGANIZATION_TYPE'].std()

u = torch.tensor(df.iloc[:, -1].values)
v = torch.tensor(df.iloc[:, :-3].values, dtype=torch.float32)
p = torch.tensor(df_prob, dtype=torch.float32).view(-1, 1)
loan = torch.tensor(df_loan, dtype=torch.float32).view(-1, 1)

In [3]:
card_u = 10
u = torch.nn.functional.one_hot(u, num_classes=card_u)
u_initial = u.argmax(dim=1)

In [ ]:
for cost_coef in [0.1, 0.15, 0.2]:
    print(cost_coef)
    stra_model = torch.load("models/stra_model_nov_pi_c{}.pkl".format(cost_coef), map_location=torch.device('cpu'))
    vani_model = torch.load("models/vani_model_pi_c{}.pkl".format(cost_coef), map_location=torch.device('cpu'))
    for model in [vani_model, stra_model]:
        _, u_category, u_shift = manipulation(u, v, card_u, model, c=cost_coef)                
        u_prime = u_category.reshape(-1)        
        print("prop manipulated: {:.4f}, prop improve: {:.4f}, mean move: {:.4f}".format(
            (u_prime!=u_initial).float().mean(), 
            (u_prime>u_initial).float().mean(),
            (u_prime-u_initial).float().mean()                
        ) )
        
    print("\n")

0.1
prop manipulated: 0.5017, prop improve: 0.5017, mean move: 1.2106
prop manipulated: 0.8883, prop improve: 0.8883, mean move: 2.1841


0.15
prop manipulated: 0.5164, prop improve: 0.5164, mean move: 1.1974
prop manipulated: 0.8740, prop improve: 0.8740, mean move: 2.0812


0.2
prop manipulated: 0.6179, prop improve: 0.6179, mean move: 1.2515
prop manipulated: 0.6697, prop improve: 0.6697, mean move: 1.2587




In [59]:
def manipulation_cutoff_oracle(u, p, loan, c=0.1):
    n = len(u)
    u_target = torch.ones((10, n))
    for i in range(10):
        u_target[i] *= i
    cate_target = cate_generate(u_target, p.reshape(1, -1), loan.reshape(1,-1))
    pi_target = (cate_target > 0).int()        
    mani_cost = c * torch.abs(u_target - u.T)
    u_shift = (pi_target - mani_cost).argmax(dim=0)
    return u_shift    
    

def cutoff_oracle(data_eval, card_u, seed, c=0.1):
    torch.manual_seed(seed)
    np.random.seed(seed)
    (u_eval, v_eval, p_eval, loan_eval) = data_eval
    # evaluation of allocation policy
    u_eval_category_shift = manipulation_cutoff_oracle(u_eval.argmax(dim=1), p_eval, loan_eval, c=c)
    tau_eval = cate_generate(u_eval_category_shift, p_eval.reshape(-1), loan_eval.reshape(-1))
    pi_eval = torch.where(tau_eval > 0, torch.tensor(1), torch.tensor(0))    
    value = torch.mean(pi_eval * tau_eval)
    return value.item()

In [ ]:
for cost_coef in [0.1, 0.15, 0.2]:
    u_shift = manipulation_cutoff_oracle(u.argmax(dim=1), p, loan, c=cost_coef)
    u_prime = u_shift

    print("prop manipulated: {:.4f}, prop improve: {:.4f}, mean move: {:.4f}".format(
        (u_prime!=u_initial).float().mean(), 
        (u_prime>u_initial).float().mean(),
        (u_prime-u_initial).float().mean()                
    ) )


prop manipulated: 0.3441, prop improve: 0.3441, mean move: 0.7853
prop manipulated: 0.3407, prop improve: 0.3407, mean move: 0.7606
prop manipulated: 0.3117, prop improve: 0.3117, mean move: 0.6063


In [63]:
for c in [0.1, 0.15, 0.2]:
    print(cutoff_oracle((u,v,p,loan), 10, 10, c=c))

1.7382351160049438
1.7350223064422607
1.7068344354629517


# Policy value


In [ ]:
for cost_coef in [0.1, 0.15, 0.2]:
    df = pd.read_csv("policy_value_curves_c{}.csv".format(cost_coef), index_col=0)

    with open('result/cutoff_{}.json'.format(cost_coef), 'r') as file:
        value_cutoff = json.load(file)  

    print(df.query("epoch==299").describe())
    print("\n")

       epoch  value_strategicGD  value_vanillaGD
count   10.0          10.000000        10.000000
mean   299.0           3.515691         2.352202
std      0.0           0.024046         0.043423
min    299.0           3.448398         2.276116
25%    299.0           3.519240         2.332728
50%    299.0           3.525430         2.348080
75%    299.0           3.525489         2.373908
max    299.0           3.525531         2.415612


       epoch  value_strategicGD  value_vanillaGD
count   10.0          10.000000        10.000000
mean   299.0           3.479063         2.285612
std      0.0           0.076027         0.031231
min    299.0           3.323268         2.225021
25%    299.0           3.514514         2.271808
50%    299.0           3.515057         2.283882
75%    299.0           3.515184         2.298754
max    299.0           3.515319         2.347664


       epoch  value_strategicGD  value_vanillaGD
count   10.0          10.000000        10.000000
mean   299.0    

In [28]:
for cost_coef in [0.1, 0.15, 0.2]:
    df = pd.read_csv("policy_value_curves_c{}.csv".format(cost_coef), index_col=0)

    stra_best_list = []
    vani_best_list = []

    for i in range(10):
        stra_best_list.append(df.iloc[i*300:(i+1)*300, 1].max())
        vani_best_list.append(df.iloc[i*300:(i+1)*300, 2].max())    

    stra_best_list = np.array(stra_best_list)    
    vani_best_list = np.array(vani_best_list)    
    print(cost_coef)
    print(stra_best_list.mean(), stra_best_list.std())
    print(vani_best_list.mean(), vani_best_list.std())
    print("\n")

0.1
3.5230405569076537 0.004380551035935266
2.6105971574783324 0.05444843442117868


0.15
3.50228955745697 0.030170446618927375
2.509178805351257 0.03949558486378261


0.2
3.1388240337371824 0.11440547452991168
2.537665367126465 0.08019133657827099


